In [1]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_311 = pd.read_csv('data/detroit-311.csv')
df_311.head()

,ticket_id,city,issue_type,ticket_status,issue_description,rating,ticket_closed_date_time,acknowledged_at,ticket_created_date_time,ticket_last_updated_date_time,address,lat,lng,location,image
0,1516722,City of Detroit,Clogged Drain,Acknowledged,"Two drains one on each side of street, street ...",3,NaN,03/06/2015 10:03:38 PM,03/06/2015 09:57:52 PM,04/12/2015 01:01:10 AM,"13120-13130 Ilene Street Detroit, MI 48238, USA",42.383998,-83.161039,"(42.3839977668, -83.1610385642)",NaN
1,1525361,City of Detroit,Clogged Drain,Acknowledged,standing water on lumplin,2,NaN,03/11/2015 04:23:11 PM,03/11/2015 04:14:29 PM,04/07/2015 02:04:44 PM,"1485 E. Outer Drive Detroit, Michigan",42.440471,-83.080919,"(42.4404708, -83.080919)",NaN
2,1525218,City of Detroit,Clogged Drain,Closed,CITZEN CALLED TO REPORT CLOGGED DRAINS,2,08/15/2015 12:03:43 AM,03/11/2015 03:39:05 PM,03/11/2015 03:26:20 PM,08/15/2015 12:03:44 AM,"15460 Eastburn Detroit, Michigan",42.445244,-82.962038,"(42.445244, -82.962038)",NaN
3,1525214,City of Detroit,Clogged Drain,Acknowledged,Citizen called DWSD to report clogged drain,3,NaN,03/11/2015 03:35:02 PM,03/11/2015 03:22:42 PM,06/07/2015 10:07:48 PM,"17541 Mendota St Detroit, Michigan",42.421043,-83.166194,"(42.421043, -83.166194)",NaN
4,1525142,City of Detroit,Clogged Drain,Acknowledged,@ THE CORNER OF GRIGGS & MARGARETA,2,NaN,03/11/2015 03:04:59 PM,03/11/2015 02:53:23 PM,03/11/2015 03:04:59 PM,"Griggs Detroit, Michigan",42.402033,-83.162874,"(42.4020334, -83.1628741)",NaN


In [3]:
def normalize_street_address(address):
    address = address.str.upper()
    address = address.str.lstrip('0')
    address = address.str.replace(r'\s+', ' ')
    address = address.str.replace(r'[^A-Z0-9&/ ]', '')
    address = address.str.replace(r'\b(STREET|ST|ROAD|RD|DRIVE|DR|AVENUE|AVE?|F & O)\b', '')
    address = address.str.replace(r' +', ' ')
    address = address.str.replace(r' DET(ROIT)? ?(MI.*|\d+)', '')
    address = address.str.replace(r' (DET(ROIT)?|MI(CHIGAN)?)$', '')
    address = address.str.replace(r'[^A-Z0-9&/ ]', '')
    address = address.str.rstrip()
    address = address.str.replace(r' (AND|&&) | ?/ ?', ' & ')
    address = address.str.replace(r' (APT|APARTMENTS?|ROOM|FLAT|BLDG?|PLANT)( ?\w?\d+)?$', '')
    return address

In [4]:
points_311 = pd.DataFrame.from_dict({
    'Lat': df_311.lat,
    'Lon': df_311.lng,
    'StreetAddress': normalize_street_address(df_311.address.map(lambda x: x.upper().split(' DETROIT,')[0])),
    'Source': '311',
    'SourceId': df_311.ticket_id,
})
points_311.head()

,Lat,Lon,Source,SourceId,StreetAddress
0,42.383998,-83.161039,311,1516722,1312013130 ILENE
1,42.440471,-83.080919,311,1525361,1485 E OUTER
2,42.445244,-82.962038,311,1525218,15460 EASTBURN
3,42.421043,-83.166194,311,1525214,17541 MENDOTA
4,42.402033,-83.162874,311,1525142,GRIGGS


In [5]:
df_violations = pd.read_csv('data/detroit-blight-violations.csv')
df_violations.head()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,TicketID,TicketNumber,AgencyName,ViolName,ViolationStreetNumber,ViolationStreetName,MailingStreetNumber,MailingStreetName,MailingCity,MailingState,...,AdminFee,LateFee,StateFee,CleanUpCost,JudgmentAmt,PaymentStatus,Void,ViolationCategory,ViolationAddress,MailingAddress
0,26288,05000001DAH,Department of Public Works,"Group, LLC, Grand Holding",2566,GRAND BLVD,743,"Beaubien, Ste. 201",Detroit,MI,...,$20.00,$150.00,$10.00,$0.00,$1680.00,PAID IN FULL,0.0,0,"2566 GRAND BLVD\nDetroit, MI\n(42.363182370000...","743 Beaubien\nDetroit, MI 48226\n(42.333730630..."
1,19800,05000025DAH,Department of Public Works,"JACKSON, RAECHELLE",19014,ASHTON,20501,HEYDEN,DETROIT,MI,...,$20.00,$10.00,$10.00,$0.00,$140.00,NO PAYMENT APPLIED,0.0,0,"19014 ASHTON\nDetroit, MI\n(42.429390762000025...","20501 HEYDEN\nDETROIT, MI 48219\n(42.442177633..."
2,19804,05000026DAH,Department of Public Works,"TALTON, CAROL ANN",18735,STAHELIN,18735,STAHELI N,DETROIT,MI,...,$20.00,$10.00,$10.00,$0.00,$140.00,PAID IN FULL,0.0,0,"18735 STAHELIN\nDetroit, MI\n(42.4287074590000...","18735 STAHELI N\nDETROIT, MI 48219\n(42.428707..."
3,20208,05000027DAH,Department of Public Works,"BONNER, DARRYL E.",20125,MONICA,25335,PEEKSKILL,SOUTHFIELD,MI,...,$20.00,$10.00,$10.00,$0.00,$140.00,NO PAYMENT APPLIED,0.0,0,"20125 MONICA\nDetroit, MI\n(42.44169828400004,...","25335 PEEKSKILL\nSOUTHFIELD, MI 48043\n(42.475..."
4,20211,05000028DAH,Department of Public Works,"GREGORY, JAMES LEE",17397,PRAIRIE,17397,PRAIRIE,DETROIT,MI,...,$20.00,$10.00,$10.00,$0.00,$140.00,PAID IN FULL,0.0,0,"17397 PRAIRIE\nDetroit, MI\n(42.42031769500005...","17397 PRAIRIE\nDETROIT, MI 48221\n(42.42031769..."


In [6]:
address_lines = df_violations.ViolationAddress.map(lambda x: x.split('\n'))
latlon = address_lines.map(lambda x: x[-1][1:-1].split(', '))
points_violations = pd.DataFrame.from_dict({
    'Lat': latlon.map(lambda x: float(x[0])),
    'Lon': latlon.map(lambda x: float(x[1])),
    'StreetAddress': normalize_street_address(address_lines.map(lambda x: x[0])),
    'Source': 'violations',
    'SourceId': df_violations.TicketID,
})
points_violations.head()

,Lat,Lon,Source,SourceId,StreetAddress
0,42.363182,-83.091677,violations,26288,2566 GRAND BLVD
1,42.429391,-83.220394,violations,19800,19014 ASHTON
2,42.428707,-83.227548,violations,19804,18735 STAHELIN
3,42.441698,-83.145018,violations,20208,20125 MONICA
4,42.420318,-83.145328,violations,20211,17397 PRAIRIE


In [7]:
df_crime = pd.read_csv('data/detroit-crime.csv', dtype={'INCINO': 'str'})
df_crime.head()

,ROWNUM,CASEID,INCINO,CATEGORY,OFFENSEDESCRIPTION,STATEOFFENSEFILECLASS,INCIDENTDATE,HOUR,SCA,PRECINCT,COUNCIL,NEIGHBORHOOD,CENSUSTRACT,ADDRESS,LON,LAT,LOCATION
0,53256,1953933,1506030028.1,ASSAULT,ASSAULT AND BATTERY/SIMPLE ASSAULT,13001.0,06/03/2015 12:00:00 AM,2,1007.0,10.0,City Council District 5,PETOSKEY-OTSEGO,5334.0,09100 PETOSKEY,-83.1221,42.3678,"PETOSKEY\n09100\n(42.3676, -83.1219)"
1,17631,1917717,1503010158.1,LARCENY,LARCENY - PARTS AND ACCESSORIES FROM VEHICLE,23006.0,03/01/2015 12:00:00 AM,9,608.0,6.0,City Council District 7,GRANDALE,5452.0,00 PLYMOUTH AND MANSFIELD,-83.2035,42.3724,"00 PLYMOUTH AND MANSFIELD\n(42.3725, -83.2033)"
2,11207,1910955,1502080223.1,STOLEN VEHICLE,VEHICLE THEFT,24001.0,02/08/2015 12:00:00 AM,18,1105.0,11.0,City Council District 3,OUTER DRIVE VAN DYKE,5051.0,00 E 7 MILE VAN DYKE,-83.0241,42.4338,"00 E 7 MILE VAN DYKE\n(42.4339, -83.0241)"
3,116589,2018186,1511090188.1,WEAPONS OFFENSES,WEAPONS OFFENSE (OTHER),52003.0,11/09/2015 12:00:00 AM,12,210.0,2.0,City Council District 6,TIREMAN,NaN,06600 BARTON,-83.1381,42.3496,"BARTON\n06600\n(42.3494, -83.1379)"
4,85790,1986862,1508239803.1,LARCENY,LARCENY - PARTS AND ACCESSORIES FROM VEHICLE,23006.0,08/14/2015 12:00:00 AM,7,309.0,3.0,City Council District 6,WEST CANFIELD,5204.0,00900 W WILLIS,-83.0692,42.3481,"W WILLIS\n00900\n(42.3481, -83.0693)"


In [35]:
points_crime = pd.DataFrame.from_dict({
    'Lat': df_crime.LAT,
    'Lon': df_crime.LON,
    'StreetAddress': normalize_street_address(df_crime.ADDRESS),
    'Source': 'crime',
    'SourceId': df_crime.INCINO,
})

# XXX: Ignore incidents without coordinates.
pred = points_crime.Lat.isnull() | points_crime.Lon.isnull()
print('dropping', pred.sum(), 'incidents')
points_crime = points_crime[~pred]
points_crime.head()
print(points_crime.size, 'incidents left')

dropping 59 incidents
599360 incidents left


In [9]:
df_demolition = pd.read_csv('data/detroit-demolition-permits.tsv', sep='\t')
df_demolition.head()

,PERMIT_NO,PERMIT_APPLIED,PERMIT_ISSUED,PERMIT_EXPIRES,SITE_ADDRESS,BETWEEN1,PARCEL_NO,LOT_NUMBER,SUBDIVISION,CASE_TYPE,...,CONTRACTOR_ADDRESS1,CONTRACTOR_ADDRESS2,CONTRACTOR_CITY,CONTRACTOR_STATE,CONTRACTOR_ZIP,CONDITION_FOR_APPROVAL,site_location,owner_location,contractor_location,geom
0,BLD2015-03955,8/28/15,8/28/15,NaN,4331 BARHAM,BETWEEN VOIGHT AND WAVENEY,21 65525-6,S15,NaN,BLD,...,13500 FOLEY,NaN,DETROIT,NaN,48227.0,NaN,"4331 BARHAM\nDetroit, MI\n(42.394106, -82.9474)","65 CADILLAC\nDETROIT, MI\n(42.331741326000042,...","13500 FOLEY\nDETROIT, MI\n(42.379332, -83.177501)",NaN
1,BLD2015-04083,8/28/15,8/28/15,NaN,9707 BESSEMORE,BETWEEN VINTON AND GRATIOT,19 2312.,40,NaN,BLD,...,13500 FOLEY,NaN,DETROIT,NaN,48227.0,NaN,"9707 BESSEMORE\nDetroit, MI\n(42.395122, -83.0...","65 CADILLAC\nDETROIT, MI\n(42.331741326000042,...","13500 FOLEY\nDETROIT, MI\n(42.379332, -83.177501)",NaN
2,BLD2015-03976,8/28/15,8/28/15,NaN,5315 BERKSHIRE,BETWEEN SOUTHAMPTON AND FRANKFORT,21 69321.,445,NaN,BLD,...,13500 FOLEY,NaN,DETROIT,NaN,48227.0,NaN,"5315 BERKSHIRE\nDetroit, MI\n(42.40322, -82.94...","65 CADILLAC\nDETROIT, MI\n(42.331741326000042,...","13500 FOLEY\nDETROIT, MI\n(42.379332, -83.177501)",NaN
3,BLD2015-03781,8/28/15,8/28/15,NaN,16670 BRINGARD DR,BETWEEN CUSHING AND SHAKESPEARE,21 25930.,97,NaN,BLD,...,13500 FOLEY,NaN,DETROIT,NaN,48227.0,NaN,"16670 BRINGARD DR\nDetroit, MI\n(42.44664, -82...","65 CADILLAC\nDETROIT, MI\n(42.331741326000042,...","13500 FOLEY\nDETROIT, MI\n(42.379332, -83.177501)",NaN
4,BLD2015-03677,8/28/15,8/28/15,NaN,1454 BEATRICE,BETWEEN TORONTO AND GILROY,20 12871.012,21,NaN,BLD,...,13500 FOLEY,NaN,DETROIT,NaN,48227.0,NaN,"1454 BEATRICE\nDetroit, MI\n(42.276956, -83.14...","65 CADILLAC\nDETROIT, MI\n(42.331741326000042,...","13500 FOLEY\nDETROIT, MI\n(42.379332, -83.177501)",NaN


In [10]:
# XXX: Ignore addresses without coordinates for now.
pred = df_demolition.site_location.str.endswith('\n', na=True)
print('dropping', pred.sum(), 'incidents')
df = df_demolition[~pred]

address_lines = df.site_location.map(lambda x: x.split('\n'))
latlon = address_lines.map(lambda x: x[-1][1:-1].split(', '))
points_demolition = pd.DataFrame.from_dict({
    'Lat': latlon.map(lambda x: float(x[0])),
    'Lon': latlon.map(lambda x: float(x[1])),
    'StreetAddress': normalize_street_address(df.SITE_ADDRESS),
    'Source': 'demolition',
    'SourceId': df.PERMIT_NO,
})
points_demolition.head()
print(points_demolition.size, 'incidents left')

dropping 817 incidents
31580 incidents left


In [34]:
points = pd.concat([
    points_311,
    points_violations,
    points_crime,
    points_demolition,
])

# XXX: Discard non-Detroit incidents.
pred = points.StreetAddress.str.contains(r'\bUSA\b')
print('dropping', pred.sum(), 'incidents')
points = points[~pred]
print(points.size, 'incidents left')

dropping 50 incidents
2268110 incidents left


In [12]:
points.sample(n=20)

,Lat,Lon,Source,SourceId,StreetAddress
20652,42.442500,-83.171200,crime,1510060427.1,20200 MEYERS
292512,42.342209,-83.039125,violations,327088,1350 SERVICE
47712,42.396978,-83.184338,violations,68637,14661 ARDMORE
39008,42.384781,-82.953299,violations,60755,3073 LAKEWOOD
88725,42.437930,-83.219323,violations,111315,19928 SOUTHFIELD
92825,42.427238,-83.218088,violations,114870,18653 FENMORE
5477,42.393800,-82.938700,crime,1507100306.1,3900 BUCKINGHAM
296438,42.358896,-83.163291,violations,329566,10418 JOY
95156,42.425900,-83.067600,crime,1509190237.1,E NEVADA & REVERE
95309,42.395000,-83.121300,crime,1506160404.1,2400 W GRAND


In [13]:
points_311.sample(10)

,Lat,Lon,Source,SourceId,StreetAddress
14665,42.343873,-83.261943,311,1837395,7350 BEAVERLAND
16635,42.426798,-83.082574,311,1897476,1810018110 DEQUINDRE
7508,42.364924,-82.932920,311,1589796,CORNER OF ALTER & KORTE
15964,42.416846,-83.258027,311,1876043,22028 GRAND RIVER
9972,42.348068,-83.204764,311,1653182,75487560 METTETAL
12685,42.333425,-83.149048,311,1773439,5711OGDEN
1120,42.402168,-82.949118,311,1688336,5264 SOMERSET
18045,42.380564,-83.097596,311,1933659,1520 LONGFELLOW
16543,42.343918,-83.070011,311,1894929,3603 MIRACLES
14958,42.347140,-83.256880,311,1844679,7637 DOLPHIN


In [14]:
points_demolition.sample(10)

,Lat,Lon,Source,SourceId,StreetAddress
3760,42.368591,-82.985807,demolition,BLD2013-01039,2597 HURLBUT
2792,42.303973,-83.129851,demolition,BLD2014-00350,8739 FALCON
5301,42.303973,-83.129851,demolition,BLD2012-03992,8739 FALCON
2734,42.404133,-82.969163,demolition,BLD2014-00555,13386 EVANSTON
1901,42.331681,-83.047996,demolition,BLD2014-04083,3038 40 PENNSYLVANIA
5588,42.395487,-82.984672,demolition,BLD2012-02897,5880 MALCOLM
3716,42.374223,-83.072250,demolition,BLD2013-01050,240 SMITH
6218,42.415189,-83.040548,demolition,BLD2011-05320,13273 DWYER
2781,42.352188,-82.999423,demolition,BLD2014-00329,1000 BALDWIN
4660,42.397151,-82.984609,demolition,BLD2012-05787,6051 LEIDICH


In [15]:
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure, show
from bokeh.models.tiles import WMTSTileSource
import pyproj

output_notebook()

Loading BokehJS ...

In [16]:
lonlat_proj = pyproj.Proj(init='epsg:4326')
xy_proj = pyproj.Proj(init='epsg:3857')

def add_xy(df):
    df = df.copy()
    if df.empty:
        return df
    xy = df.apply(lambda row: pyproj.transform(lonlat_proj, xy_proj, row.Lon, row.Lat), axis=1)
    df['X'] = xy.map(lambda row: row[0])
    df['Y'] = xy.map(lambda row: row[1])
    return df

world_topo_map = WMTSTileSource(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
    attribution='Tiles &copy; <a href="https://services.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer">ArcGIS</a>'
)

In [17]:
from bokeh.models import CategoricalColorMapper
from bokeh.plotting.helpers import DEFAULT_PALETTE
import itertools

def repeat_palette(palette, length):
    return list(itertools.islice(itertools.cycle(palette), length))

In [97]:
lon = -83.2
lat = 42.4
radius = 1 / 512 # * 4

data = points.copy()
data = data[(lon - radius < data.Lon) & (data.Lon < lon + radius) &
            (lat - radius < data.Lat) & (data.Lat < lat + radius)]
print(len(data), 'incidents')

510 incidents


In [98]:
p = figure()
p.axis.visible = False

p.add_tools(HoverTool(tooltips=[
    ('Address', '@StreetAddress'),
    ('Location', '(@Lat, @Lon)'),
    ('Incident', '@Source @SourceId'),
]))

p.circle(
    source=ColumnDataSource(data=add_xy(data)), x='X', y='Y',
    size=9, fill_color='blue', fill_alpha=0.5, line_color=None
)

p.add_tile(world_topo_map)

show(p)

In [99]:
from sklearn.cluster import DBSCAN

model = DBSCAN(eps=0.00001, min_samples=1).fit(data[['Lat', 'Lon']])
data['Label'] = model.labels_
len(data), len(set(data.Label))

(510, 239)

In [100]:
def plot_labeled(data):
    p = figure()
    p.axis.visible = False

    p.add_tools(HoverTool(tooltips=[
        ('Label', '@Label'),
        ('Address', '@StreetAddress'),
        ('Location', '(@Lat, @Lon)'),
        ('Incident', '@Source @SourceId'),
    ]))

    labels = list(data.Label.unique())

    color_mapper = CategoricalColorMapper(
        factors=labels,
        palette=repeat_palette(DEFAULT_PALETTE, len(labels))
    )

    p.circle(
        source=ColumnDataSource(data=add_xy(data)), x='X', y='Y',
        size=6,
        alpha=0.4,
        color=dict(field='Label', transform=color_mapper)
    )

    p.add_tile(world_topo_map)

    print(len(data), 'incidents')
    print(len(labels), 'points')
    show(p)
    
plot_labeled(data)

510 incidents
239 points


In [101]:
data.StreetAddress.value_counts()

15000 GREENFIELD                         28
15100 GREENFIELD                         25
15650 FENKELL                            18
15110 WINTHROP                           16
15600 FENKELL                            14
 FENKELL & GREENFIELD                    14
 GREENFIELD & FENKELL                    13
15000 WINTHROP                           12
14918 GREENFIELD                         11
14900 GREENFIELD                         11
15200 FORRER                             11
 GREENFIELD & CHALFONTE                  10
15550 FENKELL                            10
15054 FORRER                              9
14950 GREENFIELD                          9
15035 FORRER                              8
15059 FORRER                              8
15241 FORRER                              8
15020 GREENFIELD                          8
15800 FENKELL                             8
14925 GREENFIELD                          7
14950 WINTHROP                            7
15051 FORRER                    

In [102]:
for address, group in data.groupby('StreetAddress'):
    if len(group) == 1 or len(group.Label.unique()) == 1 or ' & ' in address:
        continue
    print()
    print(group)
    break


            Lat      Lon Source      SourceId StreetAddress  Label
44832   42.3985 -83.2018  crime  1501200305.1  14900 FORRER    161
67532   42.3989 -83.2009  crime  1509020320.1  14900 FORRER    187
73925   42.3982 -83.2018  crime  1504140075.1  14900 FORRER    193
94618   42.3981 -83.2016  crime  1502100313.1  14900 FORRER    210
114548  42.3984 -83.2016  crime  1508110344.1  14900 FORRER    232


In [103]:
points_crime[points_crime.StreetAddress.str.contains('15000 GREENFIELD')]

,Lat,Lon,Source,SourceId,StreetAddress
266,42.3994,-83.1991,crime,1508270077.1,15000 GREENFIELD
7935,42.4002,-83.1980,crime,1503170362.1,15000 GREENFIELD
8332,42.3994,-83.1989,crime,1510260348.1,15000 GREENFIELD
20631,42.4001,-83.1983,crime,1506270250.1,15000 GREENFIELD
21003,42.3995,-83.1988,crime,1505290197.1,15000 GREENFIELD
22360,42.3998,-83.1982,crime,1507220135.1,15000 GREENFIELD
22976,42.4004,-83.1982,crime,1506040302.1,15000 GREENFIELD
26481,42.4005,-83.1992,crime,1507150003.1,15000 GREENFIELD
28812,42.3996,-83.1990,crime,1508270091.1,15000 GREENFIELD
40118,42.3994,-83.1980,crime,1510070018.1,15000 GREENFIELD & CHALFONTE


In [104]:
sa = points.StreetAddress
is_simple_building = sa.str.contains(r'^\d+( \d+(ST|ND|RD|TH))?( [A-Z]+)*$')
is_simple_intersection = sa.str.contains(r'^( [A-Z]+)+ &( [A-Z]+)+$')
has_two_numbers = sa.str.contains(r'^\d+ \d+ ')
has_aka = sa.str.contains(' AKA ')
pd.Series(sa[~is_simple_building & ~is_simple_intersection & ~has_two_numbers & ~has_aka].unique())

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()


0                                          GRIGGS
1                                          AUBURN
2                               KEELER & SCHAEFER
3                                         DANBURY
4                  CORNER OF JOY & ROSELAWN 48204
5                                   2831 E 7 MILE
6                                        GRIXDALE
7                          DETROIT WINDSOR TUNNEL
8                                   8244 E 7 MILE
9                                       LIVERNOIS
10                           ELMWOOD CENTRAL PARK
11                                       WOODWARD
12                                    W JEFFERSON
13              WESTBOUND CHANDLER PARK AT RADNOR
14                                   620 W 7 MILE
15                                         PATTON
16                                         HOLDEN
17            CLOVERLAWN JUST NORTH OF JOY CORNER
18                          E WARREN & MARSEILLES
19                                            2ND


In [105]:
pd.Series([' STREET ST DRIVE F & O 3140 MICHIGAN']).str.replace(r'\b(STREET|ST|ROAD|RD|DRIVE|DR|AVENUE|AVE?|F & O)\b', '')

0         3140 MICHIGAN
dtype: object

In [106]:
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier

X = data.as_matrix(columns=['Lat', 'Lon'])

clf = KNeighborsClassifier(1)
#clf = RadiusNeighborsClassifier(radius=0.00001, outlier_label=-1)
clf.fit(X, data.Label)

h = 300

x_min, x_max = X[:, 0].min(), X[:, 0].max()
y_min, y_max = X[:, 1].min(), X[:, 1].max()
xx, yy = np.meshgrid(np.linspace(x_min, x_max, h),
                     np.linspace(y_min, y_max, h))
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

p = figure()
p.axis.visible = False

p.add_tools(HoverTool(tooltips=[
    ('Label', '@Label'),
    ('Address', '@StreetAddress'),
    ('Location', '(@Lat, @Lon)'),
    ('Incident', '@Source @SourceId'),
]))

labels = list(data.Label.unique())

color_mapper = CategoricalColorMapper(
    factors=labels,
    palette=repeat_palette(DEFAULT_PALETTE, len(labels))
)

x_min = add_xy(data).X.min()
x_max = add_xy(data).X.max()
y_min = add_xy(data).Y.min()
y_max = add_xy(data).Y.max()
p.image(
    image=[Z.T], x=x_min, y=y_min, dw=x_max - x_min, dh=y_max - y_min,
    alpha=0.1,
    color_mapper=color_mapper
)

p.circle(
    source=ColumnDataSource(data=add_xy(data)), x='X', y='Y',
    size=6,
    color=dict(field='Label', transform=color_mapper),
    line_color='black'
)

p.add_tile(world_topo_map)

show(p)